In [14]:

# Imports
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
# import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from tqdm import tqdm  # For nice progress bar!

import os
from torch.utils.data import DataLoader,Dataset  # Gives easier dataset managment by creating mini batches etc.
import pandas as pd
from skimage import io
from torchvision.utils import save_image

In [2]:
%%time
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
CPU times: user 1.46 s, sys: 209 ms, total: 1.67 s
Wall time: 29.6 s


In [7]:
class CatsAndDogsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)


In [8]:
# Load Data
dataset = CatsAndDogsDataset(
    csv_file="/content/drive/MyDrive/Colab Notebooks/ithome/cats_dogs.csv",
    root_dir="/content/drive/MyDrive/Colab Notebooks/ithome/cats_dogs",
    transform=transforms.ToTensor(),
)

In [52]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [53]:

# Hyperparameters
in_channel = 3
num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10


In [54]:
val_size = int(len(dataset)*0.2)
train_size = len(dataset)- int(len(dataset)*0.2)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# train_set, test_set = torch.utils.data.random_split(dataset, [5, 5])
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Model
model = torchvision.models.googlenet(pretrained=True)
model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
print(model)

In [57]:

# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

Cost at epoch 0 is 0.008119402453303337
Cost at epoch 1 is 0.004714672453701496
Cost at epoch 2 is 0.0031701961997896433
Cost at epoch 3 is 0.0026505719870328903
Cost at epoch 4 is 0.002055487595498562
Cost at epoch 5 is 0.0009487747447565198
Cost at epoch 6 is 0.0013601125683635473
Cost at epoch 7 is 0.0009867791086435318
Cost at epoch 8 is 0.0010052198776975274
Cost at epoch 9 is 0.0006522084004245698
Checking accuracy on Training Set
Got 8 / 8 with accuracy 100.00
Checking accuracy on Test Set
Got 1 / 2 with accuracy 50.00


# Data Augmentation




In [33]:

my_transforms = transforms.Compose(
    [   
        transforms.ToPILImage(),
        transforms.Resize((256, 256)),  # Resizes (256,256)
        transforms.RandomCrop((224, 224)),  # Takes a random (224,224) crop
        transforms.ColorJitter(brightness=0.5),  # Change brightness of image
        transforms.RandomGrayscale(p=0.2),  # Converts to grayscale with probability 0.2
        transforms.RandomRotation(
            degrees=45
        ),  # Perhaps a random rotation from -45 to 45 degrees
        transforms.ToTensor(),  # Finally converts PIL image to tensor so we can train w. pytorch
    ]
)


In [28]:

aug_dataset = CatsAndDogsDataset(
    csv_file="/content/drive/MyDrive/Colab Notebooks/ithome/cats_dogs.csv",
    root_dir="/content/drive/MyDrive/Colab Notebooks/ithome/cats_dogs",
    transform=my_transforms,
)

In [34]:
i = 0
for _ in range(10):
    for  img , label in aug_dataset:
        save_image(img, '/content/drive/MyDrive/Colab Notebooks/ithome/aug_cats_dogs/aug_'+str(i)+'.png')
        i+=1


# Data Augmentation - Albumentations